In [0]:
import pandas as pd
import numpy as np
import pickle
import gc
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import uniform

In [10]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [11]:
data = pickle.load(open("/content/drive/My Drive/New_data/Anadi_Basak_with_vocab/data/vocab_train.pkl", "rb"))
X_pred = (pd.read_pickle("/content/drive/My Drive/New_data/Anadi_Basak_with_vocab/data/vocab_gs_test.pkl"))

gc.collect()

168

In [12]:
print(data.shape)
print(X_pred.shape)
data.loc[:,200].value_counts()

(613890, 201)
(8916, 200)


0    524186
1     89704
Name: 200, dtype: int64

In [13]:
1 - 89704/(89704+524186)

0.8538761015817166

In [14]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split                                      #to split the dataset for training and testing
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

target = data.loc[:,200]      
data.drop(columns = [200], axis=1, inplace=True)     # taking the training data features
                                 
# del data

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.3, random_state = 32) # in this our main data is split into train and test

del data, target
gc.collect()

0

In [0]:
from sklearn.model_selection import RandomizedSearchCV
import time

In [16]:
model = RandomForestClassifier()

params_dist = {
    "n_estimators":[100],
    "min_samples_leaf": np.arange(1,60,10),
    "max_leaf_nodes": np.arange(2,60,10)
}

start_time = time.time()

optimal_model = RandomizedSearchCV(model,params_dist,scoring = "f1",n_jobs = -1)
y_pred = optimal_model.fit(X_train,y_train).predict(X_test)

stop_time = time.time()

print("Elapsed Time:", time.strftime("%H:%M:%S", time.gmtime(stop_time - start_time)))

score = f1_score(y_test,y_pred)

print("====================")
print("f1 score: {}".format(score))
print("Best Score: {}".format(optimal_model.best_score_))
print("Best Parameters: {}".format(optimal_model.best_params_))

Elapsed Time: 01:59:58
f1 score: 0.06154718854144507
Best Score: 0.059777318368905354
Best Parameters: {'n_estimators': 100, 'min_samples_leaf': 51, 'max_leaf_nodes': 32}


In [8]:
model = RandomForestClassifier()

import time

s = time.time()
model.fit(X_train, y_train)
e = time.time()

print("Model trained in", (e-s)/60,"min")
print(model.score(X_test, y_test))

pred = model.predict(X_test)
print(f1_score(pred, y_test))

Model trained in 19.38031251033147 min
0.8464545765528027
0.15567896811178789


In [0]:
cutoff = np.arange(0.005, 0.995, 0.005)
f1 = []

pred = (model.predict_proba(X_test))[:,1]

for i in cutoff:
    f1.append(f1_score(pred >= i, y_test))

# print(f1)

import plotly.graph_objects as go

fig = go.Figure(data = go.Scatter(x = cutoff, y = f1))
fig.show()

In [0]:
max_cutoff = cutoff[f1 == max(f1)]
max_cutoff

array([0.195])

In [0]:
print(accuracy_score(pred >= max_cutoff, y_test))

0.7644366254540715
